In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.attack_recipes import TextFoolerJin2019
from textattack.attacker import Attacker
from textattack.models.wrappers import ModelWrapper
import textattack

from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
from torch import nn

2022-05-12 18:26:38.895257: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
model = AutoModelForSequenceClassification.from_pretrained("bert_main_sst_model").eval()
tokenizer = AutoTokenizer.from_pretrained("bert_main_sst_model")

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)

quantized_model = torch.quantization.quantize_dynamic(
            model,{nn.Linear}, dtype=torch.qint8
        )
model_wrapper_compressed = HuggingFaceModelWrapper(quantized_model, tokenizer)

In [3]:
dataset = HuggingFaceDataset("glue", "sst2", "validation")

attack = TextFoolerJin2019.build(model_wrapper_compressed)

attack_args = textattack.AttackArgs(
    num_examples=200,
    random_seed = 42,
    query_budget = 200,
    log_to_csv ="quantized_textfooler_attacks.csv"
)
attacker = Attacker(attack, dataset,attack_args)
res_quant = attacker.attack_dataset()

Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset glue, subset sst2, split validation.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path quantized_textfooler_attacks.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       

  0%|          | 0/200 [00:00<?, ?it/s]2022-05-12 18:27:12.809361: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-12 18:27:12.833331: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-12 18:27:12.833372: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: gr003.hpc.nyu.edu
2022-05-12 18:27:12.833381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: gr003.hpc.nyu.edu
2022-05-12 18:27:12.833569: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: Invalid argument: expected %d.%d, %d.%d.%d, or %d.%d.%d.%d form for driver version; got "1"
2022-05-12 18:27:12.833595: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.57.2
2022-05-12 18:27:12.834005: I tensorflow/core/platform/cp

...

--------------------------------------------- Result 198 ---------------------------------------------
[[Negative (96%)]] --> [[Positive (87%)]]

for starters , the story is just too [[slim]] . 

for starters , the story is just too [[smart]] . 




[Succeeded / Failed / Skipped / Total] 153 / 25 / 20 / 198: 100%|█████████▉| 199/200 [2:01:50<00:36, 36.74s/it]/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 153 / 26 / 20 / 199: 100%|█████████▉| 199/200 [2:01:50<00:36, 36.74s/it]

--------------------------------------------- Result 199 ---------------------------------------------
[[Positive (97%)]] --> [[[FAILED]]]

this is a winning ensemble comedy that shows canadians can put gentle laughs and equally gentle sentiments on the button , just as easily as their counterparts anywhere else in the world . 




[Succeeded / Failed / Skipped / Total] 153 / 26 / 20 / 199: 100%|██████████| 200/200 [2:02:30<00:00, 36.75s/it]/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)
[Succeeded / Failed / Skipped / Total] 154 / 26 / 20 / 200: 100%|██████████| 200/200 [2:02:30<00:00, 36.75s/it]

--------------------------------------------- Result 200 ---------------------------------------------
[[Negative (65%)]] --> [[Positive (55%)]]

at the very least , if you do n't know anything about derrida when you walk into the [[theater]] , you [[wo]] n't know [[much]] more when you leave . 

at the very least , if you do n't know anything about derrida when you walk into the [[movies]] , you [[did]] n't know [[tremendously]] more when you leave . 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 154    |
| Number of failed attacks:     | 26     |
| Number of skipped attacks:    | 20     |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 13.0%  |
| Attack success rate:          | 85.56% |
| Average perturbed word %:     | 16.39% |
| Average num. words per input: | 17.27  |
| Avg num queries:              | 99.57  |
+-----------------------

In [4]:
import pickle
with open('textfooler_quant.pkl','wb') as f:
    pickle.dump(res_quant,f)

In [4]:
# dataset = HuggingFaceDataset("glue", "sst2", "validation")
# attack = TextFoolerJin2019.build(model_wrapper)

# attack_args = textattack.AttackArgs(
#     num_examples=200,
#     random_seed = 42,
#     query_budget = 200,
#     log_to_csv ="main_textfooler_attacks.csv"
# )
# attacker = Attacker(attack, dataset,attack_args)
# res_main = attacker.attack_dataset()

Reusing dataset glue (/home/sp6646/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

textattack: Loading datasets dataset glue, subset sst2, split validation.
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Logging to CSV at path main_textfooler_attacks.csv


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (threshold):  0.840845057
        (window_size):  15
        (skip_text_shorter_than_window):  True
        (compare_against_original):  False
      )
    (3): RepeatModification
    (4): StopwordModification
    (5): InputColumnModification(
        (matching_column_labels):  ['premise', 'hypothesis']
       


  0%|          | 0/200 [00:00<?, ?it/s]2022-05-12 18:18:07.001131: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-05-12 18:18:07.002433: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: Quadro RTX 8000 computeCapability: 7.5
coreClock: 1.62GHz coreCount: 72 deviceMemorySize: 44.49GiB deviceMemoryBandwidth: 581.23GiB/s
2022-05-12 18:18:07.002477: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-05-12 18:18:07.504866: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-05-12 18:18:07.505019: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2022-05-12 18:18:07.505045: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successful

...

--------------------------------------------- Result 198 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (92%)]]

for [[starters]] , the [[story]] is just too [[slim]] . 

for [[beginners]] , the [[storeys]] is just too [[cleverly]] . 





[Succeeded / Failed / Skipped / Total] 167 / 16 / 15 / 198: 100%|█████████▉| 199/200 [05:39<00:01,  1.71s/it]/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)

[Succeeded / Failed / Skipped / Total] 168 / 16 / 15 / 199: 100%|█████████▉| 199/200 [05:39<00:01,  1.71s/it]

--------------------------------------------- Result 199 ---------------------------------------------
[[Positive (100%)]] --> [[Negative (70%)]]

this is a [[winning]] [[ensemble]] comedy that shows [[canadians]] can put gentle laughs and equally gentle sentiments on the button , just as [[easily]] as their counterparts anywhere else in the world . 

this is a [[defeating]] [[overall]] comedy that shows [[canadiens]] can put gentle laughs and equally gentle sentiments on the button , just as [[mildly]] as their counterparts anywhere else in the world . 




/scratch/sp6646/envs_dirs/swap_env/lib/python3.8/site-packages/textattack/loggers/csv_logger.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = self.df.append(row, ignore_index=True)

[Succeeded / Failed / Skipped / Total] 169 / 16 / 15 / 200: 100%|██████████| 200/200 [05:40<00:00,  1.70s/it]


--------------------------------------------- Result 200 ---------------------------------------------
[[Negative (96%)]] --> [[Positive (58%)]]

at the very least , if you [[do]] n't know anything about derrida when you walk into the theater , you wo n't know [[much]] more when you leave . 

at the very least , if you [[suis]] n't know anything about derrida when you walk into the theater , you wo n't know [[dramatically]] more when you leave . 



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 169    |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 15     |
| Original accuracy:            | 92.5%  |
| Accuracy under attack:        | 8.0%   |
| Attack success rate:          | 91.35% |
| Average perturbed word %:     | 16.84% |
| Average num. words per input: | 17.27  |
| Avg num queries:              | 95.88  |
+-----------------------------

In [7]:
# import pickle
# with open('textfooler_main.pkl','wb') as f:
#     pickle.dump(res_main,f)